In [1]:
#檢測最新
!python --version

Python 3.13.11


In [4]:
#檢測kernel Version_system version
import sys, platform

print("Python:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("OS:", platform.system(), platform.release())
print("Machine:", platform.machine())

Python: 3.13.11 (tags/v3.13.11:6278944, Dec  5 2025, 16:26:58) [MSC v.1944 64 bit (AMD64)]
Executable: c:\Users\JianXiangWang\AppData\Local\Programs\Python\Python313\python.exe
Platform: Windows-11-10.0.26100-SP0
OS: Windows 11
Machine: AMD64


In [5]:
%pip install opencv-python
%pip install numpy
%pip install yt-dlp
%pip install line-bot-sdk
%pip install Pillow
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import cv2
import numpy as np
import csv
from datetime import datetime
from enum import Enum
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import threading
import yt_dlp
import time
from linebot import LineBotApi
from linebot.models import TextSendMessage
from linebot.exceptions import LineBotApiError

# ===== Dashboard (matplotlib) =====
try:
    import matplotlib
    from matplotlib.figure import Figure
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    import matplotlib.dates as mdates
    from matplotlib import font_manager as fm
    MATPLOTLIB_OK = True
except Exception:
    MATPLOTLIB_OK = False


def setup_chinese_font_for_matplotlib():
    """
    修正 matplotlib 中文顯示方塊問題：
    - 自動挑選系統中可用的中文字型
    - 設定 rcParams 讓中文可顯示
    """
    if not MATPLOTLIB_OK:
        return None

    candidates = [
        # Windows
        "Microsoft JhengHei", "Microsoft YaHei", "SimHei",
        # macOS
        "PingFang TC", "Heiti TC", "Arial Unicode MS",
        # Linux
        "Noto Sans CJK TC", "Noto Sans CJK SC", "Noto Sans CJK JP",
        "WenQuanYi Zen Hei", "WenQuanYi Micro Hei",
    ]

    available = {f.name for f in fm.fontManager.ttflist}
    chosen = None
    for name in candidates:
        if name in available:
            chosen = name
            break

    if chosen:
        matplotlib.rcParams["font.sans-serif"] = [chosen] + matplotlib.rcParams.get("font.sans-serif", [])
        matplotlib.rcParams["axes.unicode_minus"] = False
        return chosen

    matplotlib.rcParams["axes.unicode_minus"] = False
    return None


class WaterStatus(Enum):
    SAFE = "安全"
    WARNING = "警告"
    DANGER = "危險"
    UNKNOWN = "未知"


class VisualWaterLevelDetector:
    """視覺辨識水位偵測系統"""

    def __init__(
        self,
        warning_threshold=0.9,
        danger_threshold=1.0,
        line_token=None,
        line_user_id=None,
        line_interval=5,
        csv_root="csv_data",
    ):
        self.warning_threshold = warning_threshold
        self.danger_threshold = danger_threshold

        self.tank_top = None
        self.tank_bottom = None
        self.tank_height = None
        self.warning_level = None
        self.danger_level = None
        self.roi = None

        self.stop_processing = False
        self.last_line_notification = None
        self.line_interval = line_interval

        # LINE Bot
        self.line_token = line_token
        self.line_user_id = line_user_id
        self.line_bot = None
        if line_token:
            try:
                self.line_bot = LineBotApi(line_token)
                print("LINE Bot 初始化成功")
            except Exception as e:
                print(f"LINE Bot 初始化失敗: {e}")
                self.line_bot = None

        # CSV 分類：csv_data/yyyy-mm-dd/water_level_log_yyyymmdd_hhmmss.csv
        today_folder = datetime.now().strftime("%Y-%m-%d")
        self.csv_dir = os.path.join(csv_root, today_folder)
        os.makedirs(self.csv_dir, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.csv_filename = os.path.join(self.csv_dir, f"water_level_log_{timestamp}.csv")

        self.init_csv()

    def init_csv(self):
        with open(self.csv_filename, "w", newline="", encoding="utf-8-sig") as csvfile:
            fieldnames = [
                "時間",
                "影格編號",
                "魚缸頂端(px)",
                "水位線(px)",
                "水位高度(px)",
                "水位百分比(%)",
                "狀態",
                "備註",
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    def log_to_csv(self, frame_number, tank_top, water_line, water_level, percentage, status, note=""):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(self.csv_filename, "a", newline="", encoding="utf-8-sig") as csvfile:
            fieldnames = [
                "時間",
                "影格編號",
                "魚缸頂端(px)",
                "水位線(px)",
                "水位高度(px)",
                "水位百分比(%)",
                "狀態",
                "備註",
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(
                {
                    "時間": timestamp,
                    "影格編號": frame_number,
                    "魚缸頂端(px)": f"{tank_top:.2f}" if tank_top is not None else "N/A",
                    "水位線(px)": f"{water_line:.2f}" if water_line is not None else "N/A",
                    "水位高度(px)": f"{water_level:.2f}",
                    "水位百分比(%)": f"{percentage:.2f}",
                    "狀態": status.value,
                    "備註": note,
                }
            )

    def check_line_quota(self):
        """查詢 LINE 本月上限/已用/剩餘（若 SDK 支援）"""
        if not self.line_bot:
            return None
        try:
            quota = self.line_bot.get_message_quota()
            consumption = self.line_bot.get_message_quota_consumption()

            limit_val = getattr(quota, "value", None) or getattr(quota, "totalUsageLimit", None)
            used_val = getattr(consumption, "totalUsage", None) or getattr(consumption, "value", None)

            if limit_val is not None and used_val is not None:
                return {"limit": int(limit_val), "used": int(used_val), "remaining": int(limit_val) - int(used_val)}
            return {"quota": str(quota), "consumption": str(consumption)}
        except Exception as e:
            return {"error": str(e)}

    def send_line_notification(self, message):
        """發送LINE通知（帶間隔控制 + 429 容錯）"""
        if not self.line_bot or not self.line_user_id:
            return False

        current_time = datetime.now()
        if self.last_line_notification is not None:
            time_diff = (current_time - self.last_line_notification).total_seconds()
            if time_diff < self.line_interval:
                return False

        try:
            self.line_bot.push_message(self.line_user_id, TextSendMessage(text=message))
            self.last_line_notification = current_time
            return True
        except LineBotApiError as e:
            code = getattr(e, "status_code", None)
            print(f"LINE通知發送失敗 status={code}: {e}")
            if code == 429:
                self.last_line_notification = current_time
            return False
        except Exception as e:
            print(f"LINE通知發送失敗: {e}")
            return False

    def get_youtube_video(self, youtube_url, use_stream=True):
        """取得 YouTube 影片：優先嘗試串流，失敗則下載"""
        if use_stream:
            stream_url = self.get_youtube_stream_url(youtube_url)
            if stream_url:
                return stream_url, False

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"temp_youtube_{timestamp}.mp4"
        ydl_opts = {
            "format": "best[ext=mp4][height<=720]/best[height<=720]/best",
            "outtmpl": output_path,
            "quiet": False,
            "no_warnings": False,
        }
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([youtube_url])
            return output_path, True
        except Exception as e:
            print(f"下載失敗: {e}")
            return None, False

    def get_youtube_stream_url(self, youtube_url):
        """取得YouTube串流URL"""
        ydl_opts = {"format": "best[ext=mp4][height<=720]/best[height<=720]", "quiet": True, "no_warnings": True}
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=False)

                if "url" in info:
                    url = info["url"]
                    if "manifest" not in url.lower() and "m3u8" not in url.lower():
                        return url

                if "formats" in info:
                    for fmt in info["formats"]:
                        if fmt.get("ext") == "mp4" and fmt.get("vcodec") != "none":
                            url = fmt.get("url")
                            if url and "manifest" not in url.lower() and "m3u8" not in url.lower():
                                return url
            return None
        except Exception as e:
            print(f"取得串流URL失敗: {e}")
            return None

    def auto_detect_tank_and_water(self, frame):
        """自動偵測魚缸頂端和水位線（邊緣檢測 + 輪廓）"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 30, 100)

        kernel = np.ones((3, 3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        edges = cv2.erode(edges, kernel, iterations=1)

        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)

            self.roi = (x, y, w, h)
            self.tank_top = y
            self.tank_bottom = y + h

            roi_frame = frame[y : y + h, x : x + w]
            water_line_y = self.detect_water_line_in_roi(roi_frame)

            water_line_absolute = (y + water_line_y) if water_line_y is not None else None
            return self.tank_top, self.tank_bottom, water_line_absolute

        return None, None, None

    def detect_water_line_in_roi(self, roi_frame):
        """在ROI中偵測水位線（Hough + 顏色變化 fallback）"""
        h, w = roi_frame.shape[:2]
        gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)

        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=int(w * 0.3), maxLineGap=10)

        water_surface_y = None
        if lines is not None:
            horizontal_lines = []
            for line in lines:
                x1, y1, x2, y2 = line[0]
                if abs(y2 - y1) < 10:
                    horizontal_lines.append((y1 + y2) / 2)
            if horizontal_lines:
                water_surface_y = min(horizontal_lines)

        if water_surface_y is None:
            max_diff = 0
            best_y = None
            for i in range(20, h - 20, 5):
                upper_mean = np.mean(gray[max(0, i - 15) : i, :])
                lower_mean = np.mean(gray[i : min(h, i + 15), :])
                diff = abs(upper_mean - lower_mean)

                if diff > max_diff and diff > 15:
                    max_diff = diff
                    best_y = i
            water_surface_y = best_y

        return water_surface_y

    def detect_water_level(self, frame):
        tank_top, tank_bottom, water_line = self.auto_detect_tank_and_water(frame)
        if tank_top is not None and tank_bottom is not None:
            self.tank_height = tank_bottom - tank_top
            self.warning_level = self.tank_height * self.warning_threshold
            self.danger_level = self.tank_height * self.danger_threshold

            water_level = (tank_bottom - water_line) if water_line is not None else 0
            return tank_top, water_line, water_level

        return None, None, 0

    def check_status(self, water_level):
        if self.tank_height is None or water_level == 0:
            return WaterStatus.UNKNOWN
        if water_level >= self.danger_level:
            return WaterStatus.DANGER
        if water_level >= self.warning_level:
            return WaterStatus.WARNING
        return WaterStatus.SAFE

    def draw_overlay(self, frame, tank_top, water_line, water_level, status):
        if self.roi is not None:
            x, y, w, h = self.roi
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)

            if tank_top is not None:
                cv2.line(frame, (x, int(tank_top)), (x + w, int(tank_top)), (255, 0, 0), 2)

            if water_line is not None:
                cv2.line(frame, (x, int(water_line)), (x + w, int(water_line)), (0, 255, 255), 2)

        percentage = (water_level / self.tank_height) * 100 if self.tank_height else 0

        if status == WaterStatus.DANGER:
            color = (0, 0, 255)
            note = "危險！"
        elif status == WaterStatus.WARNING:
            color = (0, 165, 255)
            note = "警告！"
        elif status == WaterStatus.SAFE:
            color = (0, 255, 0)
            note = "正常"
        else:
            color = (128, 128, 128)
            note = "偵測中"

        cv2.putText(
            frame,
            f"Water Level: {water_level:.1f}px ({percentage:.1f}%)",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2,
        )
        cv2.putText(
            frame,
            f"Status: {status.value} - {note}",
            (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            color,
            2,
        )
        return frame

    def process_video(self, video_source, is_youtube=False, is_camera=False, progress_callback=None):
        """
        處理：
        - local file
        - youtube url
        - camera index
        """
        self.stop_processing = False
        self.last_line_notification = None

        temp_file = None
        is_temp_file = False

        if is_camera:
            camera_index = int(video_source)
            if os.name == "nt":
                cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)
            else:
                cap = cv2.VideoCapture(camera_index)
            video_path = f"camera:{camera_index}"
        else:
            if is_youtube:
                result = self.get_youtube_video(video_source, use_stream=True)
                if result[0] is None:
                    if progress_callback:
                        progress_callback("錯誤：無法取得YouTube影片")
                    return
                video_path, is_temp_file = result
                if is_temp_file:
                    temp_file = video_path
            else:
                video_path = video_source

            cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            if progress_callback:
                progress_callback("錯誤：無法開啟影片/鏡頭")
            return

        if progress_callback:
            progress_callback(f"來源: {video_path}")
            progress_callback(f"CSV記錄檔: {self.csv_filename}")

            if self.line_bot and self.line_user_id:
                quota = self.check_line_quota()
                if quota and "limit" in quota:
                    progress_callback(f"LINE 本月額度: {quota['used']}/{quota['limit']}，剩餘 {quota['remaining']}")
                elif quota and "error" in quota:
                    progress_callback(f"LINE 額度查詢失敗: {quota['error']}")

        frame_number = 0
        paused = False
        display_frame = None

        while True:
            if self.stop_processing:
                break

            if not paused:
                ret, frame = cap.read()
                if not ret:
                    if progress_callback:
                        progress_callback("影片處理完成 / 鏡頭已中斷")
                    break

                frame_number += 1

                tank_top, water_line, water_level = self.detect_water_level(frame)
                status = self.check_status(water_level)
                percentage = (water_level / self.tank_height) * 100 if self.tank_height else 0

                # CSV 每 10 幀記一次
                if frame_number % 10 == 0:
                    note = ""
                    if status == WaterStatus.DANGER:
                        note = "水位過高，請立即處理！"
                    elif status == WaterStatus.WARNING:
                        note = "水位接近上限，請注意！"
                    self.log_to_csv(frame_number, tank_top, water_line, water_level, percentage, status, note)

                    if progress_callback and frame_number % 100 == 0:
                        progress_callback(f"已處理: {frame_number} 影格｜狀態: {status.value}｜水位: {percentage:.1f}%")

                # LINE 通知（警告/危險）
                if status in (WaterStatus.WARNING, WaterStatus.DANGER):
                    msg = (
                        f"⚠️ 注意魚缸溢出安全！\n"
                        f"當前水位: {percentage:.1f}%\n"
                        f"狀態: {status.value}\n"
                        f"時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
                    )
                    self.send_line_notification(msg)

                display_frame = self.draw_overlay(frame.copy(), tank_top, water_line, water_level, status)

            if display_frame is not None:
                cv2.imshow("Water Level Detection", display_frame)

            key = cv2.waitKey(30) & 0xFF
            if key == ord("q"):
                break
            elif key == ord("p"):
                paused = not paused

        cap.release()
        cv2.destroyAllWindows()

        if temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except Exception:
                pass

        if progress_callback:
            progress_callback(f"記錄已儲存至: {self.csv_filename}")


class WaterLevelGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("智能魚缸水位偵測系統")
        self.root.geometry("1400x820")
        self.root.minsize(1200, 720)

        self.detector = None
        self.processing_thread = None
        self.last_csv_path = None

        # matplotlib 中文字型設定
        self.mpl_font = setup_chinese_font_for_matplotlib()

        # Ocean style
        self.apply_ocean_style()

        self.create_widgets()
        self.refresh_dashboard_sources()

    def apply_ocean_style(self):
        # Ocean palette
        self.COL_BG = "#061a2e"         # 深海
        self.COL_PANEL = "#0b2b4a"      # 面板
        self.COL_TEXT = "#e8f7ff"       # 文字
        self.COL_MUTED = "#b7d9ff"      # 次要文字
        self.COL_ACCENT = "#00b4d8"     # 點綴
        self.COL_BTN = "#0077b6"        # 按鈕
        self.COL_BTN_HOVER = "#0096c7"  # hover

        self.root.configure(bg=self.COL_BG)

        if os.name == "nt":
            base_font = ("Microsoft JhengHei", 10)
            title_font = ("Microsoft JhengHei", 16, "bold")
        else:
            base_font = ("Noto Sans CJK TC", 10)
            title_font = ("Noto Sans CJK TC", 16, "bold")

        self.base_font = base_font
        self.title_font = title_font
        self.root.option_add("*Font", base_font)

        style = ttk.Style()
        try:
            style.theme_use("clam")
        except Exception:
            pass

        style.configure("TFrame", background=self.COL_BG)
        style.configure("TLabelframe", background=self.COL_BG, foreground=self.COL_TEXT)
        style.configure("TLabelframe.Label", background=self.COL_BG, foreground=self.COL_ACCENT, font=(base_font[0], 11, "bold"))

        style.configure("TLabel", background=self.COL_BG, foreground=self.COL_TEXT)
        style.configure("TNotebook", background=self.COL_BG, borderwidth=0)
        style.configure("TNotebook.Tab", background=self.COL_PANEL, foreground=self.COL_TEXT, padding=(10, 6))
        style.map("TNotebook.Tab", background=[("selected", self.COL_ACCENT)], foreground=[("selected", "#002233")])

        style.configure("TEntry", fieldbackground="#ffffff", foreground="#001219")
        style.configure("TCombobox", fieldbackground="#ffffff", foreground="#001219")

        style.configure("TButton", background=self.COL_BTN, foreground="white", padding=(10, 6), borderwidth=0)
        style.map("TButton", background=[("active", self.COL_BTN_HOVER), ("disabled", "#1b4965")], foreground=[("disabled", "#d0e8ff")])

    # ======= 右側狀態/儀表板佈局（PanedWindow 左控制 / 右顯示） =======
    def create_widgets(self):
        title_label = tk.Label(self.root, text="智能魚缸水位偵測系統", font=self.title_font, bg=self.COL_BG, fg=self.COL_TEXT)
        title_label.pack(pady=(10, 6))

        main_pane = ttk.Panedwindow(self.root, orient="horizontal")
        main_pane.pack(fill="both", expand=True, padx=14, pady=(0, 14))

        left_panel = ttk.Frame(main_pane)
        right_panel = ttk.Frame(main_pane)

        main_pane.add(left_panel, weight=1)
        main_pane.add(right_panel, weight=3)

        def _set_sash():
            try:
                main_pane.sashpos(0, 420)
            except Exception:
                pass
        self.root.after(200, _set_sash)

        # ===== 左側：控制區 =====
        source_frame = ttk.LabelFrame(left_panel, text="影片來源", padding=10)
        source_frame.pack(fill="x", padx=10, pady=(10, 8))

        self.source_type = tk.StringVar(value="local")
        ttk.Radiobutton(source_frame, text="本地影片檔案", variable=self.source_type, value="local", command=self.update_source_ui).pack(anchor="w")
        ttk.Radiobutton(source_frame, text="YouTube URL", variable=self.source_type, value="youtube", command=self.update_source_ui).pack(anchor="w")
        ttk.Radiobutton(source_frame, text="本地鏡頭", variable=self.source_type, value="camera", command=self.update_source_ui).pack(anchor="w")

        input_frame = ttk.Frame(left_panel)
        input_frame.pack(fill="x", padx=10, pady=8)

        self.path_label = ttk.Label(input_frame, text="影片路徑:")
        self.path_label.pack(anchor="w")

        path_input_frame = ttk.Frame(input_frame)
        path_input_frame.pack(fill="x", pady=(4, 0))

        self.path_entry = ttk.Entry(path_input_frame)
        self.path_entry.pack(side="left", fill="x", expand=True)

        self.browse_button = ttk.Button(path_input_frame, text="瀏覽", command=self.browse_file)
        self.browse_button.pack(side="right", padx=(6, 0))

        cam_frame = ttk.Frame(left_panel)
        cam_frame.pack(fill="x", padx=10, pady=(0, 8))
        self.cam_label = ttk.Label(cam_frame, text="鏡頭索引 (0:預設, 1:USB camera):")
        self.cam_label.pack(side="left")
        self.cam_index_entry = ttk.Entry(cam_frame, width=10)
        self.cam_index_entry.insert(0, "0")
        self.cam_index_entry.pack(side="left", padx=8)

        line_frame = ttk.LabelFrame(left_panel, text="LINE 通知設定 (選填)", padding=10)
        line_frame.pack(fill="x", padx=10, pady=8)

        ttk.Label(line_frame, text="Channel Access Token:").grid(row=0, column=0, sticky="w", pady=5)
        self.line_token_entry = ttk.Entry(line_frame, width=46)
        self.line_token_entry.grid(row=0, column=1, sticky="we", pady=5, padx=5)

        ttk.Label(line_frame, text="User ID:").grid(row=1, column=0, sticky="w", pady=5)
        self.line_user_entry = ttk.Entry(line_frame, width=46)
        self.line_user_entry.grid(row=1, column=1, sticky="we", pady=5, padx=5)

        ttk.Label(line_frame, text="通知間隔 (秒):").grid(row=2, column=0, sticky="w", pady=5)
        self.line_interval_entry = ttk.Entry(line_frame, width=10)
        self.line_interval_entry.insert(0, "5")
        self.line_interval_entry.grid(row=2, column=1, sticky="w", pady=5, padx=5)

        self.test_line_button = ttk.Button(line_frame, text="測試LINE", command=self.test_line)
        self.test_line_button.grid(row=2, column=1, sticky="e", pady=5, padx=5)

        line_frame.columnconfigure(1, weight=1)

        param_frame = ttk.LabelFrame(left_panel, text="偵測參數設定", padding=10)
        param_frame.pack(fill="x", padx=10, pady=8)

        ttk.Label(param_frame, text="警告閾值 (%):").grid(row=0, column=0, sticky="w", pady=5)
        self.warning_entry = ttk.Entry(param_frame, width=10)
        self.warning_entry.insert(0, "90")
        self.warning_entry.grid(row=0, column=1, sticky="w", pady=5)

        ttk.Label(param_frame, text="危險閾值 (%):").grid(row=1, column=0, sticky="w", pady=5)
        self.danger_entry = ttk.Entry(param_frame, width=10)
        self.danger_entry.insert(0, "100")
        self.danger_entry.grid(row=1, column=1, sticky="w", pady=5)

        button_frame = ttk.Frame(left_panel)
        button_frame.pack(fill="x", padx=10, pady=(8, 10))

        self.start_button = ttk.Button(button_frame, text="開始分析", command=self.start_processing)
        self.start_button.pack(side="left", padx=5)

        self.stop_button = ttk.Button(button_frame, text="停止", command=self.stop_processing, state="disabled")
        self.stop_button.pack(side="left", padx=5)

        ttk.Frame(left_panel).pack(fill="both", expand=True)

        # ===== 右側：狀態 / 儀表板 =====
        status_frame = ttk.LabelFrame(right_panel, text="狀態 / 儀表板", padding=10)
        status_frame.pack(fill="both", expand=True, padx=10, pady=10)

        self.notebook = ttk.Notebook(status_frame)
        self.notebook.pack(fill="both", expand=True)

        # Tab 1: 狀態訊息
        self.log_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.log_tab, text="狀態訊息")

        self.status_text = tk.Text(
            self.log_tab,
            state="disabled",
            bg=self.COL_PANEL,
            fg=self.COL_TEXT,
            insertbackground=self.COL_TEXT,
            relief="flat",
        )
        self.status_text.pack(side="left", fill="both", expand=True)

        scrollbar = ttk.Scrollbar(self.log_tab, command=self.status_text.yview)
        scrollbar.pack(side="right", fill="y")
        self.status_text.config(yscrollcommand=scrollbar.set)

        # Tab 2: 儀表板
        self.dash_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.dash_tab, text="儀表板")

        dash_top = ttk.Frame(self.dash_tab)
        dash_top.pack(fill="x", pady=(8, 6))

        ttk.Label(dash_top, text="日期資料夾:").pack(side="left")
        self.date_combo = ttk.Combobox(dash_top, width=14, state="readonly")
        self.date_combo.pack(side="left", padx=6)

        self.refresh_dash_button = ttk.Button(dash_top, text="重新整理列表", command=self.refresh_dashboard_sources)
        self.refresh_dash_button.pack(side="left", padx=6)

        self.load_latest_button = ttk.Button(dash_top, text="載入最新", command=self.load_latest_csv_to_dashboard)
        self.load_latest_button.pack(side="left", padx=6)

        list_frame = ttk.Frame(self.dash_tab)
        list_frame.pack(fill="x", padx=2, pady=(0, 6))

        ttk.Label(list_frame, text="CSV 檔案（可多選合併）:").pack(anchor="w")

        lb_frame = ttk.Frame(list_frame)
        lb_frame.pack(fill="x", pady=(4, 0))

        self.csv_listbox = tk.Listbox(
            lb_frame,
            selectmode="extended",
            height=7,
            bg="#ffffff",
            fg="#001219",
            relief="flat",
        )
        self.csv_listbox.pack(side="left", fill="x", expand=True)

        lb_scroll = ttk.Scrollbar(lb_frame, command=self.csv_listbox.yview)
        lb_scroll.pack(side="right", fill="y")
        self.csv_listbox.config(yscrollcommand=lb_scroll.set)

        btn_row = ttk.Frame(self.dash_tab)
        btn_row.pack(fill="x", pady=(0, 6))

        self.btn_select_all = ttk.Button(btn_row, text="全選", command=self.select_all_csv)
        self.btn_select_all.pack(side="left", padx=4)

        self.btn_clear_sel = ttk.Button(btn_row, text="取消全選", command=self.clear_csv_selection)
        self.btn_clear_sel.pack(side="left", padx=4)

        self.btn_load_sel = ttk.Button(btn_row, text="載入選取(繪圖)", command=self.load_selected_csvs_to_dashboard)
        self.btn_load_sel.pack(side="left", padx=4)

        self.dash_info = ttk.Label(self.dash_tab, text="")
        self.dash_info.pack(anchor="w", pady=(0, 6))

        self.plot_container = ttk.Frame(self.dash_tab)
        self.plot_container.pack(fill="both", expand=True)

        self.figure = None
        self.canvas = None

        self.date_combo.bind("<<ComboboxSelected>>", lambda e: self.update_csv_file_list())
        self.update_source_ui()

        if MATPLOTLIB_OK and self.mpl_font is None:
            self.log_status("⚠️ 找不到可用中文字型，圖表中文可能仍顯示方塊。")

    def update_source_ui(self):
        st = self.source_type.get()
        if st == "local":
            self.path_label.config(text="影片路徑:")
            self.browse_button.config(state="normal")
            self.path_entry.config(state="normal")
            self.cam_label.config(state="disabled")
            self.cam_index_entry.config(state="disabled")
        elif st == "youtube":
            self.path_label.config(text="YouTube URL:")
            self.browse_button.config(state="disabled")
            self.path_entry.config(state="normal")
            self.cam_label.config(state="disabled")
            self.cam_index_entry.config(state="disabled")
        else:
            self.path_label.config(text="（鏡頭模式不需填路徑）")
            self.browse_button.config(state="disabled")
            self.path_entry.delete(0, tk.END)
            self.path_entry.config(state="disabled")
            self.cam_label.config(state="normal")
            self.cam_index_entry.config(state="normal")

    def browse_file(self):
        filename = filedialog.askopenfilename(
            title="選擇影片檔案",
            filetypes=[("影片檔案", "*.mp4 *.avi *.mov *.mkv"), ("所有檔案", "*.*")],
        )
        if filename:
            self.path_entry.delete(0, tk.END)
            self.path_entry.insert(0, filename)

    def log_status(self, message):
        """thread-safe UI 更新"""
        if threading.current_thread() is not threading.main_thread():
            self.root.after(0, lambda: self.log_status(message))
            return

        self.status_text.config(state="normal")
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.status_text.insert(tk.END, f"[{timestamp}] {message}\n")
        self.status_text.see(tk.END)
        self.status_text.config(state="disabled")

    def test_line(self):
        token = self.line_token_entry.get().strip()
        uid = self.line_user_entry.get().strip()
        if not token or not uid:
            messagebox.showwarning("提醒", "請先填入 Channel Access Token 與 User ID")
            return

        def worker():
            try:
                bot = LineBotApi(token)

                # quota / consumption
                remaining = None
                try:
                    q = bot.get_message_quota()
                    c = bot.get_message_quota_consumption()
                    limit_val = getattr(q, "value", None) or getattr(q, "totalUsageLimit", None)
                    used_val = getattr(c, "totalUsage", None) or getattr(c, "value", None)
                    if limit_val is not None and used_val is not None:
                        remaining = int(limit_val) - int(used_val)
                        self.log_status(f"[LINE測試] 本月額度：{used_val}/{limit_val}，剩餘 {remaining}")
                        if remaining <= 0:
                            self.log_status("[LINE測試] 剩餘額度不足，略過推播測試")
                            return
                except Exception as e:
                    self.log_status(f"[LINE測試] 額度查詢失敗：{e}")

                bot.push_message(uid, TextSendMessage(text="✅ 智能魚缸水位偵測系統：LINE 推播測試成功"))
                self.log_status("[LINE測試] 推播成功 ✅")

            except LineBotApiError as e:
                self.log_status(f"[LINE測試] 推播失敗 status={getattr(e, 'status_code', 'N/A')}: {e}")
            except Exception as e:
                self.log_status(f"[LINE測試] 推播失敗: {e}")

        threading.Thread(target=worker, daemon=True).start()

    def start_processing(self):
        st = self.source_type.get()

        if st != "camera":
            video_path = self.path_entry.get().strip()
            if not video_path:
                messagebox.showerror("錯誤", "請輸入影片路徑或YouTube URL")
                return
        else:
            try:
                cam_index = int(self.cam_index_entry.get().strip() or "0")
            except ValueError:
                messagebox.showerror("錯誤", "鏡頭索引必須是數字 (0/1/2...)")
                return
            video_path = cam_index

        try:
            warning_threshold = float(self.warning_entry.get()) / 100
            danger_threshold = float(self.danger_entry.get()) / 100
            line_interval = int(self.line_interval_entry.get())
        except ValueError:
            messagebox.showerror("錯誤", "閾值和間隔時間必須是數字")
            return

        line_token = self.line_token_entry.get().strip() or None
        line_user_id = self.line_user_entry.get().strip() or None

        self.detector = VisualWaterLevelDetector(
            warning_threshold=warning_threshold,
            danger_threshold=danger_threshold,
            line_token=line_token,
            line_user_id=line_user_id,
            line_interval=line_interval,
            csv_root="csv_data",
        )

        self.start_button.config(state="disabled")
        self.stop_button.config(state="normal")

        self.log_status("=" * 60)
        self.log_status("開始處理...")
        self.log_status(f"警告閾值: {warning_threshold * 100:.1f}%")
        self.log_status(f"危險閾值: {danger_threshold * 100:.1f}%")
        self.log_status(f"CSV輸出: {self.detector.csv_filename}")

        is_youtube = (st == "youtube")
        is_camera = (st == "camera")

        def process():
            self.detector.process_video(video_path, is_youtube=is_youtube, is_camera=is_camera, progress_callback=self.log_status)
            self.last_csv_path = self.detector.csv_filename
            self.root.after(0, self.processing_complete)

        self.processing_thread = threading.Thread(target=process, daemon=True)
        self.processing_thread.start()

    def stop_processing(self):
        if self.detector:
            self.detector.stop_processing = True
        self.log_status("正在停止...")

    def processing_complete(self):
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")
        self.log_status("處理完成！")

        self.refresh_dashboard_sources()
        self.load_latest_csv_to_dashboard()

    # ===== Dashboard: 資料來源列表 =====
    def refresh_dashboard_sources(self):
        base = "csv_data"
        os.makedirs(base, exist_ok=True)

        dates = []
        for name in os.listdir(base):
            p = os.path.join(base, name)
            if os.path.isdir(p):
                dates.append(name)
        dates.sort(reverse=True)

        self.date_combo["values"] = dates
        if dates:
            if not self.date_combo.get():
                self.date_combo.set(dates[0])
            self.update_csv_file_list()
        else:
            self.csv_listbox.delete(0, tk.END)
            self.date_combo.set("")
            self.dash_info.config(text="找不到任何 csv_data 紀錄。")

    def update_csv_file_list(self):
        date_folder = self.date_combo.get().strip()
        if not date_folder:
            return
        folder = os.path.join("csv_data", date_folder)
        if not os.path.isdir(folder):
            return

        files = [f for f in os.listdir(folder) if f.lower().endswith(".csv")]
        files.sort(reverse=True)

        self.csv_listbox.delete(0, tk.END)
        for f in files:
            self.csv_listbox.insert(tk.END, f)

        if files:
            self.csv_listbox.selection_clear(0, tk.END)
            self.csv_listbox.selection_set(0)

    def select_all_csv(self):
        self.csv_listbox.selection_set(0, tk.END)

    def clear_csv_selection(self):
        self.csv_listbox.selection_clear(0, tk.END)

    def get_selected_csv_paths(self):
        date_folder = self.date_combo.get().strip()
        if not date_folder:
            return []
        folder = os.path.join("csv_data", date_folder)
        idxs = list(self.csv_listbox.curselection())
        if not idxs:
            return []
        names = [self.csv_listbox.get(i) for i in idxs]
        return [os.path.join(folder, n) for n in names]

    def load_selected_csvs_to_dashboard(self):
        paths = self.get_selected_csv_paths()
        if not paths:
            messagebox.showwarning("提醒", "請先在列表選擇一個或多個 CSV")
            return
        self.plot_csvs(paths)

    def load_latest_csv_to_dashboard(self):
        if self.last_csv_path and os.path.exists(self.last_csv_path):
            self.plot_csvs([self.last_csv_path])
            return
        paths = self.get_selected_csv_paths()
        if paths:
            self.plot_csvs([paths[0]])

    # ===== Dashboard: 讀取/合併/繪圖 =====
    def read_csv_to_series(self, csv_path):
        times, percentages, statuses = [], [], []
        status_map = {"安全": 0, "警告": 1, "危險": 2, "未知": -1}

        with open(csv_path, "r", encoding="utf-8-sig") as f:
            reader = csv.DictReader(f)
            for row in reader:
                t = (row.get("時間") or "").strip()
                p = (row.get("水位百分比(%)") or "").strip()
                s = (row.get("狀態") or "").strip()
                if not t or not p:
                    continue
                try:
                    dt = datetime.strptime(t, "%Y-%m-%d %H:%M:%S")
                    pv = float(p)
                    sv = status_map.get(s, -1)
                    times.append(dt)
                    percentages.append(pv)
                    statuses.append(sv)
                except Exception:
                    continue

        return times, percentages, statuses

    def merge_series(self, list_of_paths):
        # 合併多 CSV -> 依時間排序，簡單去重（同時間取最後一筆）
        merged = {}
        for p in list_of_paths:
            t, perc, st = self.read_csv_to_series(p)
            for i in range(len(t)):
                merged[t[i]] = (perc[i], st[i])

        times = sorted(merged.keys())
        percentages = [merged[x][0] for x in times]
        statuses = [merged[x][1] for x in times]
        return times, percentages, statuses

    def status_segments(self, times, statuses):
        """把狀態序列切成連續區段 (t_start, t_end, status_val)"""
        if not times:
            return []
        segs = []
        start_i = 0
        for i in range(1, len(times)):
            if statuses[i] != statuses[i - 1]:
                segs.append((times[start_i], times[i - 1], statuses[i - 1]))
                start_i = i
        segs.append((times[start_i], times[-1], statuses[-1]))
        return segs

    def plot_csvs(self, csv_paths):
        if not MATPLOTLIB_OK:
            self.dash_info.config(text="matplotlib 尚未安裝，無法顯示圖表。請先 pip install matplotlib")
            return

        if len(csv_paths) == 1:
            times, percentages, statuses = self.read_csv_to_series(csv_paths[0])
            label = os.path.basename(csv_paths[0])
        else:
            times, percentages, statuses = self.merge_series(csv_paths)
            label = f"合併 {len(csv_paths)} 檔（一天趨勢）"

        if not times:
            self.dash_info.config(text="CSV 無有效資料，請確認檔案內容。")
            return

        total = len(statuses)
        count_unknown = sum(1 for s in statuses if s == -1)
        count_safe = sum(1 for s in statuses if s == 0)
        count_warn = sum(1 for s in statuses if s == 1)
        count_danger = sum(1 for s in statuses if s == 2)

        for w in self.plot_container.winfo_children():
            w.destroy()

        seg_color = {
            -1: "#adb5bd",  # unknown
            0: "#2ec4b6",   # safe
            1: "#ffb703",   # warning
            2: "#fb5607",   # danger
        }
        line_color = "#48cae4"
        grid_color = "#7aaed6"

        self.figure = Figure(figsize=(10.2, 5.2), dpi=100)
        gs = self.figure.add_gridspec(1, 2, width_ratios=[3, 1])

        ax = self.figure.add_subplot(gs[0, 0])
        ax_pie = self.figure.add_subplot(gs[0, 1])

        self.figure.patch.set_facecolor(self.COL_BG)
        ax.set_facecolor(self.COL_PANEL)
        ax_pie.set_facecolor(self.COL_PANEL)

        # 依狀態底色區段
        for (t0, t1, sv) in self.status_segments(times, statuses):
            ax.axvspan(t0, t1, color=seg_color.get(sv, "#adb5bd"), alpha=0.14, linewidth=0)

        ax.plot(times, percentages, linewidth=1.8, color=line_color)
        ax.set_title("水位百分比歷史趨勢（含狀態區段）", color=self.COL_TEXT)
        ax.set_xlabel("時間", color=self.COL_TEXT)
        ax.set_ylabel("水位百分比(%)", color=self.COL_TEXT)
        ax.grid(True, alpha=0.18, color=grid_color)
        ax.tick_params(axis="x", colors=self.COL_MUTED)
        ax.tick_params(axis="y", colors=self.COL_MUTED)
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
        self.figure.autofmt_xdate()

        # 圓餅圖：警告/危險比例
        warn_danger_total = count_warn + count_danger
        ax_pie.set_title("警告/危險比例", color=self.COL_TEXT)
        if warn_danger_total == 0:
            ax_pie.text(0.5, 0.5, "無警告/危險資料", ha="center", va="center", color=self.COL_TEXT, transform=ax_pie.transAxes)
            ax_pie.pie([1, 1], colors=[seg_color[1], seg_color[2]], startangle=90)
        else:
            ax_pie.pie(
                [count_warn, count_danger],
                labels=["警告", "危險"],
                colors=[seg_color[1], seg_color[2]],
                autopct="%1.1f%%",
                startangle=90,
                textprops={"color": self.COL_TEXT},
            )

        inv_status = {-1: "未知", 0: "安全", 1: "警告", 2: "危險"}
        last_status_str = inv_status.get(statuses[-1], "未知")
        last_percent = percentages[-1]

        self.dash_info.config(
            text=f"已載入：{label} ｜ 筆數：{total} ｜ 最新狀態：{last_status_str} ｜ 最新水位：{last_percent:.1f}% ｜ "
                 f"安全:{count_safe} 警告:{count_warn} 危險:{count_danger} 未知:{count_unknown}"
        )

        self.canvas = FigureCanvasTkAgg(self.figure, master=self.plot_container)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(fill="both", expand=True)


if __name__ == "__main__":
    root = tk.Tk()
    app = WaterLevelGUI(root)
    root.mainloop()
